# Credits

This is heavily influenced or copied from https://github.com/pytorch/tutorials


# Autograd: automatic differentiation

Central to all neural networks in PyTorch is the `autograd` package.
Let’s first briefly visit this, and we will then go to training our first neural network.

The `autograd` package provides automatic differentiation for all operations on Tensors.
It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Let us see this in more simple terms with some examples.


## 1. Tensor

`torch.Tensor` is the central class of the package. Setting the attribute `.requires_grad` to `True` will make the tensor "record" all operations on it. When you finish your computation you can call `.backward()` and have all the gradients computed automatically. The gradient for this tensor will be accumulated into the `.grad` attribute.

![autograd.Variable](../static_files/autograd-variable.png)

There’s one more class which is very important for autograd implementation - a `Function`.

`Tensor` and `Function` are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each tensor has a `.grad_fn` attribute that references a `Function` that has created the `Tensor` (except for Tensors created by the user - their `grad_fn` is `None`).

If you want to compute the derivatives, you can call `.backward()` on a Tensor. If `Tensor` is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to backward(), however if it has more elements, you need to specify a `gradient` argument that is a tensor of matching shape.


In [20]:
import torch

Create a tensor


In [21]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Do a tensor operation:


In [22]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


`y` was created as a result of an operation, so it has a `grad_fn`.


In [23]:
print(y.grad_fn)

Do more operations on y


In [24]:
z = y * y * 3
out = z.mean()

print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


# Assignments

1. Create a Tensor that `requires_grad` of size (5, 5)
2. Sum the values in the Tensor


In [25]:
tensor_grad = torch.rand(5, 5, requires_grad=True)
print(tensor_grad)

tensor_sum = torch.sum(tensor_grad)
print(tensor_sum)

tensor([[0.3353, 0.3297, 0.3591, 0.0970, 0.3433],
        [0.9307, 0.2622, 0.8614, 0.3831, 0.2803],
        [0.6910, 0.6187, 0.5142, 0.7140, 0.7962],
        [0.4479, 0.8917, 0.8287, 0.5200, 0.0119],
        [0.6852, 0.6776, 0.0717, 0.0110, 0.9773]], requires_grad=True)
tensor(12.6393, grad_fn=<SumBackward0>)


## 2. Gradients

Let’s backprop now. Because `out` contains a single scalar, `out.backward()` is equivalent to `out.backward(torch.tensor([1.0]))`


In [26]:
out.backward()

Print gradients d(out)/dx


In [27]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


You should have a matrix of `4.5`. Let’s denote the tensor `out` with $o$.

We have:
$o = \frac{1}{4}\sum_i z_i$, $z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.

Therefore, $\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$,
hence $\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.

You can do many crazy things with autograd!


In [28]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([1513.2335, -717.3866, -876.2304], grad_fn=<MulBackward0>)


In [29]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


**Read later** \
_Documentation_ \
`Tensor`: https://pytorch.org/docs/stable/tensors.html \
`Function`: http://pytorch.org/docs/autograd


# Assignments

1. Define a tensor and set `requires_grad` to `True`
2. Multiply the tensor by 2 and assign the result to a new python variable (i.e. `x = result`)
3. Sum the variable's elements and assign to a new python variable
4. Print the gradients of all the variables
5. Now perform a backward pass on the last variable (NOTE: for each new python variable that you define, call `.retain_grad()`)
6. Print all gradients again

- what did you notice?


In [31]:
import warnings

# Ignores the warning (ignored for readability):
# UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed
warnings.filterwarnings("ignore")


# Leaf tensor (defined by user)
# Final stop of the flow of gradients on the backward pass
tensor_with_grad = torch.rand(5, 5, requires_grad=True)
print(tensor_with_grad)

# Tensors defined using mathematical operations
tensor_mul = tensor_with_grad * 2
print(tensor_mul)

tensor_sum = torch.sum(tensor_mul)
print(tensor_sum)

# Gradient functions
# print("Gradient function of tensor_with_grad:", tensor_with_grad.grad_fn)
# print("Gradients function of tensor_mul:", tensor_mul.grad_fn)
# print("Gradients function of tensor_sum:", tensor_sum.grad_fn)

print()

print("Gradients before backward pass")
print("Gradients of tensor_with_grad:", tensor_with_grad.grad)
print("Gradients of tensor_mul:", tensor_mul.grad)
print("Gradients of tensor_sum:", tensor_sum.grad)

print()

# Retains intermediate gradients
tensor_mul.retain_grad()
tensor_sum.retain_grad()

# Backward pass
tensor_sum.backward()

print("Gradients after backward pass")
print("Gradients of tensor_with_grad:", tensor_with_grad.grad)
print("Gradients of tensor_mul:", tensor_mul.grad)
print("Gradients of tensor_sum:", tensor_sum.grad)

tensor([[0.2794, 0.8622, 0.0066, 0.7495, 0.3338],
        [0.9157, 0.4034, 0.3200, 0.1980, 0.4721],
        [0.0538, 0.6208, 0.8647, 0.6263, 0.0585],
        [0.8412, 0.3079, 0.2139, 0.7655, 0.2053],
        [0.4687, 0.7702, 0.3993, 0.1157, 0.9373]], requires_grad=True)
tensor([[0.5589, 1.7244, 0.0131, 1.4990, 0.6676],
        [1.8314, 0.8069, 0.6399, 0.3959, 0.9442],
        [0.1075, 1.2416, 1.7295, 1.2527, 0.1171],
        [1.6824, 0.6158, 0.4277, 1.5311, 0.4106],
        [0.9374, 1.5403, 0.7986, 0.2315, 1.8745]], grad_fn=<MulBackward0>)
tensor(23.5797, grad_fn=<SumBackward0>)

Gradients before backward pass
Gradients of tensor_with_grad: None
Gradients of tensor_mul: None
Gradients of tensor_sum: None

Gradients after backward pass
Gradients of tensor_with_grad: tensor([[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]])
Gradients of tensor_mul: tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1